In [33]:
import os
import pprint
#from BCBio import GFF
import pandas

In [34]:
## none of these are currently being used
#baf_gff = 'genomes/bafPKo.gff'
#bb31_gff = 'genomes/bbB31.gff'
#bga_gff = 'genomes/bgaPBi.gff'

In [35]:
print(os.getcwd())

/home/mf019/longread_pangenome/expanded_dataset_analysis/group2BB/notebook


In [58]:
rs2bb = '/home/mf019/longread_pangenome/expanded_dataset_analysis/group2BB/B31_refseq_id_to_old_name.tsv'
rs2bb_df = pandas.read_csv(rs2bb, sep='\t')
rs2bb_df = rs2bb_df.rename(columns={
    "refseq_id": "gene",
    "old_name": "old_locus_tag"
})
rs2bb_df['old_locus_tag'] = rs2bb_df['old_locus_tag'].apply(lambda x: x.split('%2C')[0])

In [36]:
sp_pangenome_ref = '/home/mf019/longread_pangenome/expanded_dataset_analysis/output/results/v8/v8_filtered_sp/roary_v8_filtered_sp/pan_genome_reference.fa'
nsp_pangenome_ref = '/home/mf019/longread_pangenome/expanded_dataset_analysis/output/results/v8/v8_filtered_nsp/roary_v8_filtered_nsp/pan_genome_reference.fa'

In [37]:
# set blast results
v8_sp_blast_results = '/home/mf019/longread_pangenome/expanded_dataset_analysis/output/results/v8/v8_filtered_sp/alignments/pangenome_vs_B31.topHit.tsv'
v8_nsp_blast_results = '/home/mf019/longread_pangenome/expanded_dataset_analysis/output/results/v8/v8_filtered_nsp/alignments/pangenome_vs_B31.topHit.tsv'

In [60]:
ver = 'v8'
typeof = ['filtered_sp', 'filtered_nsp']

for t in typeof:
    g2idmap = f'/home/mf019/longread_pangenome/expanded_dataset_analysis/output/results/{ver}/{ver}_{t}/alignments/{ver}_{t}_group_to_ID_mapping.txt'
    g2BBmap = f'/home/mf019/longread_pangenome/expanded_dataset_analysis/output/results/{ver}/{ver}_{t}/alignments/{ver}_{t}_groupID_to_BBgene.tsv'
    mg_out = f'/home/mf019/longread_pangenome/expanded_dataset_analysis/output/results/{ver}/{ver}_{t}/alignments/{ver}_{t}_missing_groups.tsv'
    pg_ref = f'/home/mf019/longread_pangenome/expanded_dataset_analysis/output/results/{ver}/{ver}_{t}/roary_{ver}_{t}/pan_genome_reference.fa'
    blast_results = f'/home/mf019/longread_pangenome/expanded_dataset_analysis/output/results/{ver}/{ver}_{t}/alignments/pangenome_vs_B31.topHit.tsv'
    
    if not os.path.exists(g2idmap):
        # Create the group-to-ID mapping
        with open(pg_ref) as f, open(g2idmap, 'w') as out:
            for line in f:
                if line.startswith('>'):
                    line = line.replace('>', '').split(' ')
                    seqID = line[0]
                    groupID = ' '.join(line[1:])
                    if 'BB' in groupID:
                        groupID = 'BB'.join(groupID.split('BB'))
                    out.write(f'{seqID}\t{groupID}\n')

    # Read BLAST results
    results = pandas.read_csv(blast_results, sep='\t', header=None)
    print("results:", results)

    # Read ID-to-group mapping
    id2group = pandas.read_csv(g2idmap, sep='\t', header=None, names=["ID", "group"])
    print("id2group:", id2group)

    # Blast TSV Header: 
    #(0, 'query_acc.ver') (1, 'subject_acc.ver') (2, '%_identity') (3, 'alignment') 
    #(4, 'length') (5, 'mismatches') (6, 'gap_opens') (7, 'q._start') 
    #(8, 'q._end') (9, 's._start') (10, 's._end') (11, 'evalue') (12, 'bit_score')
    # Create the hits DataFrame
    hits = pandas.DataFrame({
        "ID": results[0],
        "gene": results[1],
        "percent_ident": results[2],
        "alignment_length": results[3],
        "E-score": results[10]
    })
    hits['gene'] = hits['gene'].apply(lambda x: x.strip('gene-'))
    # Merge hits with id2group to map groups
    hits = hits.merge(id2group, on="ID", how="left")

    # Drop duplicates
    hits = hits.drop_duplicates()

    hits = hits.merge(rs2bb_df, on="gene", how="left")
    
    # Filter for genes that start with "gene-BB"
    #hits = hits.loc[hits["gene"].astype(str).str.startswith("gene-BB")]

    # Sort and group by query ID
    top_hits = hits.sort_values(
        by=["ID", "group", "gene", "old_locus_tag", "E-score", "percent_ident", "alignment_length"],
        ascending=[True, True, True, False, False, False, False]
    ).groupby("ID", as_index=False).first()
        # Handle duplicate columns (group_x and group_y)
    if "group_y" in top_hits.columns:
        top_hits = top_hits.drop(columns=["group_y"]).rename(columns={"group_x": "group"})
    print("top_hits:", top_hits)
    # Reorder columns
    translatedhits = top_hits[["ID", "group", "gene", "old_locus_tag", "percent_ident", "alignment_length", "E-score"]]
    print("translatedhits:", translatedhits)
    # Find groups in id2group that are not in translatedhits
    missing_groups = id2group[~id2group["group"].isin(translatedhits["group"])]

    # Output missing groups
    print("Missing Groups:")
    print(missing_groups)
    # Write the final results to a TSV file
    translatedhits.to_csv(g2BBmap, sep='\t', header=True, index=False)
    missing_groups.to_csv(mg_out, sep='\t', header=True, index=False)

results:                  0                1        2     3    4   5    6     7    8   \
0      CFHKPG_00001  gene-BB_RS00005  100.000   564    0   0    1   564   10   
1      CFHKPG_00002  gene-BB_RS00010  100.000  1029    0   0    1  1029    1   
2      CFHKPG_00003  gene-BB_RS00015   99.932  1467    0   1    1  1467    1   
3      CFHKPG_00004  gene-BB_RS00020  100.000  1794    0   0    1  1794    1   
4      CFHKPG_00005  gene-BB_RS00025  100.000  1056    0   0    1  1056    1   
...             ...              ...      ...   ...  ...  ..  ...   ...  ...   
27507  PHNMIL_01551  gene-BB_RS06635   70.404   446  120   5    1   440  127   
27508  PHNMIL_01551  gene-BB_RS05785   69.802   404  114   4   41   440  161   
27509  PHNMIL_01551  gene-BB_RS04455   69.974   383  111   4   61   441  184   
27510  PHNMIL_01551  gene-BB_RS05530   67.025   279   82   5  119   392  197   
27511  PHNMIL_01551  gene-BB_RS07780   77.215    79   18   0  230   308   98   

         9             10     